In [ ]:
## load in all of the libraries we need
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import psutil
from itertools import islice 

## load in the model
model = torch.hub.load('mtangemann/deepgazemr', 'DeepGazeMR', pretrained=True)

In [ ]:
## This section trims the video down into smaller videos.
## This is currently done to save memory - but I would like
## to be able to run this code on larger videos and skip 
## this step entirely

# Open the large video file for reading
input_video = cv2.VideoCapture('Video_Game.mp4')

# Get video properties (width, height, frames per second, etc.)
frame_width = int(input_video.get(3))  # Width
frame_height = int(input_video.get(4))  # Height
fps = int(input_video.get(5))  # Frames per second
total_frames = int(input_video.get(7))  # Total number of frames

# Define the number of smaller videos you want to create
num_videos = 4

# Specify the output folder for the smaller videos
output_folder = 'smaller_videos_4chunks'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Calculate the number of frames per smaller video
frames_per_video = total_frames // num_videos
print(total_frames)
print(num_videos)
print(frames_per_video)
print(frame_width)
print(frame_height)

# Loop through and create smaller videos
for i in range(num_videos):
    # Create a VideoWriter object for the smaller video
    output_video = cv2.VideoWriter(os.path.join(output_folder, f'smaller_video_{i + 1}.mp4'), 
                                   cv2.VideoWriter_fourcc(*'mp4v'), 
                                   fps, 
                                   (frame_width, frame_height))
    
    # Write frames to the smaller video
    for j in range(frames_per_video):
        ret, frame = input_video.read()
        if ret:
            output_video.write(frame)
        else:
            break
    
    # Release the smaller video file
    output_video.release()

# Release the large video file
input_video.release()

# Close all OpenCV windows (if any)
cv2.destroyAllWindows()


In [ ]:
## Currently using cpu as the torch device because my CPU 
## has more memory than my GPU. Ideally this could be running
## on Beocat with a large amount of GPU memory

#if torch.cuda.is_available():
    #device = torch.device('cuda')
    ## Check the available GPU memory
    #gpu_properties = torch.cuda.get_device_properties(device)
    #gpu_memory = gpu_properties.total_memory / (1024**2)  # in megabytes
    #print(f"GPU Name: {gpu_properties.name}")
    #print(f"GPU Memory: {gpu_memory} MB")
#else:
    #print("CUDA is not available. Using CPU.")

cpu_memory = psutil.virtual_memory().available / (1024**2)  # in megabytes
print(f"CPU Memory: {cpu_memory} MB")

#device = torch.device('cuda')
device = torch.device('cpu')
model.to(device)


In [ ]:
## Load up the video, and get the pixel information from 
## each frame into a torch object

import skvideo.io

video = skvideo.io.vread('smaller_videos_4chunks/smaller_video_2.mp4')
video = torch.from_numpy(video).type(torch.float32)
video = video.permute(0, 3, 1, 2) / 255.0
video = video.to(device)

In [ ]:
## set output directory
output_directory = 'heatmap_frames_4chunks'
os.makedirs(output_directory, exist_ok=True)

## run the model on all frames (running on the "video" object 
## until there are no more frames), and save the outputs to 
## individual numpy files. These numpy files will be read by
## VideoCreation.ipynb
with torch.no_grad(): #torch.no_grad was added to hopefully reduce the amount of memory being used.
    for i, prediction in enumerate(model.predict(video)):
        if prediction is not None:
            filename = f'{output_directory}/prediction_{i + vidframenumstart}.npy'
            np.save(filename, prediction.cpu().numpy())

            print(f"Saved prediction {i} to {filename}")
